In [1]:
# data_path = 'E:/GitHub/CS-5302-Project-Group-15/Datasets/'

import os 
# PATH = 'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/'
# os.chdir(PATH)
from IPython.display import Markdown
from python_scripts import llm_rag
from python_scripts import machine_translation
from python_scripts import text_to_speech
from python_scripts import whisper_setup
from python_scripts import llm_rag
from python_scripts import get_audio
from python_scripts import utils

PATH = os.getcwd().replace('\\', '/')
root_path = PATH + '/Datasets/MeDAL' # maybe CAPS 'data_path' too
data_path = root_path + '/training_data(2).jsonl'


# import replicate

# deployment = replicate.deployments.get("ubaidtariq8/llama2-med-genai")
# prediction = deployment.predictions.create(
#   input={"prompt": "..."}
# )
# prediction.wait()
# print(prediction.output)

In [2]:
# Not an exhaustive list
models = { 
    'llama_ours' : 'ubaidtariq8/llama2-med-genai',
    'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
    'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
    'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
}

In [ ]:
model = llm_rag.DocumentEmbeddingPipeline(
    model_version = models['llama_ours'], chroma_path = root_path # change this to experiment with different models available on replicate
)
model.setup_environment()
model.prepare_documents(data_path = data_path, collection_name = "medal_filtered",  joining = False, persistent = True)
model.embed_and_index()